## Data Loaders

This notebook demonstrates the use of **graph loader** in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.



### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [1]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

from pyTigerGraph.datasets import Datasets

dataset = Datasets("Cora")

conn.ingestDataset(dataset, getToken=config["getToken"])

from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

A folder with name Cora already exists in ./tmp. Skip downloading.
---- Checking database ----
A graph with name Cora already exists in the database. Skip ingestion.
Graph name is set to Cora for this connection.


CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

### Graph Loader

`GraphLoader` pulls (possibly disconnected) subgraphs from database. Different from NeighborLoader which produces connected subgraphs, this loader gets (random) batches of edges and vertices attached to those edges.

**Note**: For the first time you initialize the loader on a graph in TigerGraph, the initialization might take a minute as it installs the corresponding query to the database and optimizes it. However, the query installation only needs to be done once, so it will take no time when you initialize the loader on the same TG graph again.
        
There are two ways to use the data loader. See [here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all data at once (`num_batches=1`), there will be only one batch (of all the data) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader itself.

Args:
* v_in_feats (list or dict, optional):
        Vertex attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* v_out_labels (list or dict, optional):
        Vertex attributes to be used as labels for prediction. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* v_extra_feats (list or dict, optional):
        Other attributes to get such as indicators of train/test data.
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type. 
        All types of attributes are allowed. Defaults to None.
* e_in_feats (list or dict, optional):
        Edge attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* e_out_labels (list or dict, optional):
        Edge attributes to be used as labels for prediction. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* e_extra_feats (list or dict, optional):
        Other edge attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        All types of attributes are allowed. Defaults to None.
* batch_size (int, optional):
        Number of edges in each batch.
        Defaults to None.
* num_batches (int, optional):
        Number of batches to split the edges.
        Defaults to 1.
* shuffle (bool, optional):
        Whether to shuffle the data before loading.
        Defaults to False.
* filter_by (str, optional):
        A boolean attribute used to indicate which edges can be included.
        Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader.
        Only "PyG", "DGL" and "dataframe" are supported. Defaults to "dataframe".
* add_self_loop (bool, optional):
        Whether to add self-loops to the graph. Defaults to False.
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

# Testcase1: using graphLoader with callback_fn to get batches of graphs.(for homogeneous graph)    
## Results: run successfully, data loaded completely

In [3]:
def process_batch(batch):
    return batch
graph_loader2 = conn.gds.graphLoader(
    num_batches=10,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    callback_fn = process_batch
)
for i, batch in enumerate(graph_loader2):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 1037], edge_feat=[1037], is_train=[1037], is_val=[1037], x=[1008, 1433], y=[1008], train_mask=[1008], val_mask=[1008], test_mask=[1008])
----Batch 1----
Data(edge_index=[2, 1114], edge_feat=[1114], is_train=[1114], is_val=[1114], x=[1280, 1433], y=[1280], train_mask=[1280], val_mask=[1280], test_mask=[1280])
----Batch 2----
Data(edge_index=[2, 1095], edge_feat=[1095], is_train=[1095], is_val=[1095], x=[1279, 1433], y=[1279], train_mask=[1279], val_mask=[1279], test_mask=[1279])
----Batch 3----
Data(edge_index=[2, 1111], edge_feat=[1111], is_train=[1111], is_val=[1111], x=[1307, 1433], y=[1307], train_mask=[1307], val_mask=[1307], test_mask=[1307])
----Batch 4----
Data(edge_index=[2, 1047], edge_feat=[1047], is_train=[1047], is_val=[1047], x=[1164, 1433], y=[1164], train_mask=[1164], val_mask=[1164], test_mask=[1164])
----Batch 5----
Data(edge_index=[2, 987], edge_feat=[987], is_train=[987], is_val=[987], x=[972, 1433], y=[972], train_mask=[972], val_

# Testcase2: using graphLoader with callback_fn to get batchs of graphs(for heterogeneous graph).  
## case details:using graphLoader without callback_fn first, then using graphLoader with the same paprams but set a callback_fn to get part of data.
## Results: run successfully, data loaded completely

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [4]:
conn.graphname="hetero"

# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

('6g7cvva0rt33rf8s8mprt24dockhlidp', 1674821647, '2023-01-27 12:14:07')

In [6]:
loader3 = conn.gds.graphLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    e_extra_feats={"v0v0": ["is_train", "is_val"],
                    "v1v1": ["is_train", "is_val"]},
    batch_size=1024,
    shuffle=False,
    filter_by=None,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4
)
for i, batch in enumerate(loader3):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
HeteroData(
  v0={
    x=[76, 231],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[110, 171] },
  (v0, v0v0, v0)={
    edge_index=[2, 412],
    is_train=[412],
    is_val=[412]
  },
  (v1, v1v1, v1)={
    edge_index=[2, 565],
    is_train=[565],
    is_val=[565]
  }
)
----Batch 1----
HeteroData(
  v0={
    x=[76, 231],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[110, 171] },
  (v0, v0v0, v0)={
    edge_index=[2, 298],
    is_train=[298],
    is_val=[298]
  },
  (v1, v1v1, v1)={
    edge_index=[2, 479],
    is_train=[479],
    is_val=[479]
  }
)


In [9]:
def process_batch(batch):
    return {"v0":batch["v0"]}
loader4 = conn.gds.graphLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    e_extra_feats={"v0v0": ["is_train", "is_val"],
                    "v1v1": ["is_train", "is_val"]},
    batch_size=1024,
    shuffle=False,
    filter_by=None,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4,
    callback_fn=process_batch
)
for i, batch in enumerate(loader4):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
{'v0': {'x': tensor([[-0.6769,  0.0808,  0.3906,  ..., -1.3234,  2.0952, -0.2452],
        [-1.0063, -0.7678,  0.1999,  ...,  0.5784, -0.8078,  0.9916],
        [-0.7545,  0.6728,  0.1221,  ...,  1.5865,  1.4805, -0.9357],
        ...,
        [-1.2434, -0.6484,  0.6525,  ...,  2.1990, -0.5989,  1.5437],
        [-0.7531,  1.9722,  1.3593,  ..., -0.4019,  0.5499, -0.7318],
        [ 0.6978, -0.3825,  0.8817,  ..., -0.7247, -0.2079, -2.5429]],
       dtype=torch.float64), 'y': tensor([5, 7, 8, 2, 7, 9, 4, 7, 7, 1, 6, 3, 8, 1, 4, 7, 4, 9, 2, 4, 1, 9, 6, 4,
        8, 5, 8, 4, 8, 1, 7, 6, 7, 0, 7, 6, 5, 3, 1, 5, 9, 2, 3, 9, 3, 8, 1, 4,
        4, 9, 9, 1, 8, 7, 5, 0, 6, 0, 1, 0, 4, 6, 4, 9, 9, 7, 8, 7, 1, 1, 1, 1,
        2, 9, 1, 2]), 'train_mask': tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, 

# Testcase3: using Loader with callback_fn to loaddata(via Kafka).  
## Results: run successfully, data loaded completely

**Note**: Kafka streaming function is only available for the Enterprise Edition. You need to activate the Enterprise Edition to use it. 

In [10]:
conn.graphname="Cora"
# COMMENT OUT THE LINE BELOW if you are NOT using a graph that requires token authentication
conn.getToken(conn.createSecret())

('0qsr8kg4r8kd0a6u4dlq4ue5k2vskaia', 1674821736, '2023-01-27 12:15:36')

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official [doc](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) for detailed settings.

In [11]:
conn.gds.configureKafka(kafka_address ="your_Kafka_address")

#### Get batches of vertices

In [12]:
def process_batch(batch):
    return batch
graph_loader5 = conn.gds.graphLoader(
    num_batches=10,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None,
    callback_fn = process_batch
)
for i, batch in enumerate(graph_loader5):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 1050], edge_feat=[1050], is_train=[1050], is_val=[1050], x=[1022, 1433], y=[1022], train_mask=[1022], val_mask=[1022], test_mask=[1022])
----Batch 1----
Data(edge_index=[2, 1051], edge_feat=[1051], is_train=[1051], is_val=[1051], x=[1264, 1433], y=[1264], train_mask=[1264], val_mask=[1264], test_mask=[1264])
----Batch 2----
Data(edge_index=[2, 1014], edge_feat=[1014], is_train=[1014], is_val=[1014], x=[1209, 1433], y=[1209], train_mask=[1209], val_mask=[1209], test_mask=[1209])
----Batch 3----
Data(edge_index=[2, 1036], edge_feat=[1036], is_train=[1036], is_val=[1036], x=[1239, 1433], y=[1239], train_mask=[1239], val_mask=[1239], test_mask=[1239])
----Batch 4----
Data(edge_index=[2, 1065], edge_feat=[1065], is_train=[1065], is_val=[1065], x=[1163, 1433], y=[1163], train_mask=[1163], val_mask=[1163], test_mask=[1163])
----Batch 5----
Data(edge_index=[2, 1119], edge_feat=[1119], is_train=[1119], is_val=[1119], x=[1057, 1433], y=[1057], train_mask=[1057